#1. Define The Problem

When we do PCR to amplify our libraries, we will end up with many copies of reads. Using certain signatures found in a SAM file we can identify these PCR duplicates and eliminate them. If we do not do this, certain analyses, such as gene expression, can be altered by the presence of the same read over and over again as it will just be interpretted as a very abundant transcript, thus messing up the analysis.


#2. Write examples

Example Input:

1:497:R:-272+13M17D24M:ATCGTTG	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37


19:20389:F:275+18M2D19M:ATCGTTG	99	1	497	0	37M	=	17919	314	CGAGTCTGACCTGAGGAGAAGTGTGCTCCGCATTCAG	>>>>>>>>>>>>>>>>>>>><<>>><<>>4::>>:<9	RG:Z:UM0098:1	XT:A:R	NM:i:0	SM:i:0	AM:i:0	X0:i:4	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37


19:20389:F:275+18M2D19M:ATCGGTG	147	1	499	0	2S35M	=	17644	-314	TTGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	;44999;499<8<8<<<8<<><<<<><7<;<<<>><<	XT:A:R	NM:i:2	SM:i:0	AM:i:0	X0:i:4	X1:i:0	XM:i:0	XO:i:1	XG:i:2	MD:Z:18^CA19


9:21597+10M2I25M:R:-209:TTGAACT	83	1	21678	0	8M2I27M	=	21469	-244	CACCACATCACATATACCAAGCCTGGCTGTGTCTTCT	<;9<<5><<<<><<<>><<><>><9>><>>>9>>><>	XT:A:R	NM:i:2	SM:i:0	AM:i:0	X0:i:5	X1:i:0	XM:i:0	XO:i:1	XG:i:2	MD:Z:35

Example Output:

1:497:R:-272+13M17D24M	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37

9:21597+10M2I25M:R:-209	83	1	21678	0	8M2I27M	=	21469	-244	CACCACATCACATATACCAAGCCTGGCTGTGTCTTCT	<;9<<5><<<<><<<>><<><>><9>><>>>9>>><>	XT:A:R	NM:i:2	SM:i:0	AM:i:0	X0:i:5	X1:i:0	XM:i:0	XO:i:1	XG:i:2	MD:Z:35


We got rid of the second and third read because the second read has almost the same sequence, the same start position, and the same UMI as the first read, thus it is a PCR duplicate, and the third read has the same read after the first two bases and has soft clipping that accounts for those first two being off, as well as having a very similar UMI, so it is also a PCR duplicate. The fourth read is completely different so it is not a PCR duplicate and will remain.

#3. Pseudocode

Notes:

- Want bitwise flag to determine strandedness
- don't need sequnce
- need corrected start position, UMI, and strand
- If reverse strand want to check end position (technically where that strand started)
- For paired end reads, need to a double check that both pairs are not PCR duplicates
- Look at sam file to see how much you need to save at a time

In [6]:
#Run samtools sort on SAM file and feed ouput to following Python script

#Initiate an empty dictionary
#Open SAM file
    #Use readline to read each line of the file one at a time
    #run isolate_UMI to look specifically at the UMI
    #If we have a list of known UMIs check if UMI is in list and toss line if it is not
    #Check if UMI is in dictionary
        #If no, add line from file to dictionary as a value, where the key is the UMI
            #write this line to an output file that will contain no PCR duplicates
        #If yes, run check_PCR_dup on all lines in the dictionary that are values for that key
            #If check_PCR_dup is false then it is not a PCR duplicate
                #We then want to add this file line as a value to the UMI key already in the dictionary
                #Also write this line to the same output file 
            #If check_PCR_dup is true then it is a PCR duplicate
                #Can write duplicates to a separate output file if so desired
                #Now we want to ignore the PCR duplicate so we just move on to the next line of the file
    #Now we need to flush the dictionary every so often so it doesn't get too big 
    #To make sure we get all of the PCR duplicates, we don't want to flush the dictionary too early
    #Create a variable that saves the length of the line from the file
    #Rewrite this variable if the line you are reading is longer than the previously saved variable
    #If the start position of a line is 2 times this saved variable away from the previously saved lines in the dictionary,
    #flush the dictionary because since the file is sorted by start position PCR duplicates cannot possibly occur that 
    #far away from each other.
    
    #Stop reading through the file once it has been fully read through
    
    #For paired-end reads, do the same thing except look at 2 lines of file at a time because the paired end file is formatted
    #such that there is a read and then its read pair directly after it
    #For paired reads to be PCR duplicates, both the read and read pair have to match up with a previously discovered read, 
    #read-pair combo.

#4. High Level Functions

In [8]:
def start_pos_check(string1,string2):
    '''Checks the start position of two reads seeing if they're matching'''
    #Isolate column containing start position
    #Check cigar string for soft clipping
    #If there is soft clipping, subtract amount of soft clipping from start position 
    #After adjusting for soft clipping, see if start position matches
    #Return True if match, False if no match
    #Example input: string 1 start = 100 CIGAR = 100M, string 2 start = 110 CIGAR = 10S90M
    #Example output: True
    
def isolate_UMI(string1, string2):
    '''Checks if UMIs are matching'''
    #Use regex to isolate umi from qname
    #Return UMI
    #Example input: 1:497:R:-272+13M17D24M:ATCGTTG
    #Example ouput: ATCGTTG
    
def check_chrom(string1, string2):
    '''Checks if read comes from the same chromosome'''
    #Use regex to isolate chromosome from rname
    #Checks to see if from same chromosome
    #Return True if match, False if no match
    #Example input: 1:497:R:-272+13M17D24M:ATCGTTG	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37
    #Example input: 1:497:R:-272+13M17D24M:ATCGTTG	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37
    #Example output: True (because the RNAME for both is 1)

def end_pos_check(string1,string2):
    '''Checks end postion of two reads to see if they're matching'''
    #Isolate column containing start position
    #Check cigar string for soft clipping
    #If there is soft clipping, subtract amount of soft clipping from start position 
    #Now add the number next to the M to the start position to find the end position
    #Return True if match, False is no match
    #Example input: Line with start position of 100 CIGAR string of 100M
    #Example input: Line with start position of 100 CIGAR sting of 10S80M10S
    #Example output: False (input 1 has end position 200, input 2 has end position 190)
    
def strand_check(string):
    '''Checks if reads came from same strand of DNA'''
    #Isolate column containing bitwise flag
    #Decode bitwise flag to look for strandedness flag 
    #Return "+" if forward, "-" if reverse.
    #Example Input: 1:497:R:-272+13M17D24M:ATCGTTG	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37
    #Example Input: 1:497:R:-272+13M17D24M:ATCGTTG	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37
    #Example Output: True (both are +)
    
def check_PCR_dup(string1,string2):
    '''Checks if a PCR duplicate is present. Returns True if they are PCR duplicates, False if they are not'''
    if check_chrom(string1,string2) == True:
        if strand_check(string1)==strand_check(string2):
            if strand_check(string1) == "+":
                if start_pos_check(string1,string2) == True:
                    return True
                else:
                    return False
            if strand_check(string1) == "-":
                if end_pos_check(string1,string2) == True:
                    return True
                else:
                    return False
        else:
            return False
    else:
        return False
    
    #Example Input: 1:497:R:-272+13M17D24M:ATCGTTG	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37
    #Example Input: 1:497:R:-272+13M17D24M:ATCGTTG	113	1	497	37	37M	15	100338662	0	CGGGTCTGACCTGAGGAGAACTGTGCTCCGCCTTCAG	0;==-==9;>>>>>=>>>>>>>>>>>=>>>>>>>>>>	XT:A:U	NM:i:0	SM:i:37	AM:i:0	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:37
    #Example Output: True
        